Analyse propka files to get other structure metrics

Folded and unfolded pI

In [21]:
import os
import numpy as np
#from multiprocessing import Pool
import pandas as pd
import json
from operator import itemgetter
from collections import defaultdict, Counter
import freesasa
import copy

In [22]:
dataset_name = 'vhh_twist'

In [23]:
# path = '/storage/evagsm/nobackup/crystal_dataset/structures/ABB_paired_propka/'
# files = [path + f for f in os.listdir(path) if f.endswith('.pka')] 
# print(len(files))

#propka_path = '/vols/opig/users/gordon/tnp_project/greiff_results/'+dataset_name+'_propka/'
propka_path = '/data/localhost/gordon/TNP_Project/RESULTS/greiff_results/propka/'+dataset_name+'_propka/'
propka_files = [propka_path + f for f in os.listdir(propka_path) if f.endswith('.pka')] 

In [24]:
len(propka_files)

108

In [25]:
def run_pi(file):
    try:
        with open(file) as rf:
            pI_line = [line for line in rf.readlines() if line.startswith('The pI is')][0]
            #print(pI_line.split()[3], pI_line.split()[6])
            return pI_line.split()[3], pI_line.split()[6]
    except Exception:
        return None, None

In [26]:
all_results = dict()
for f in propka_files:
    structure = f.split('/')[-1].split('.')[0]
    results = run_pi(f)
    all_results[structure] = results

In [27]:
len(all_results)

108

In [28]:
df = pd.DataFrame(all_results).T.reset_index()
df.columns = ['SeqID','folded_pI', 'unfolded_pI']

In [29]:
df.head()

,SeqID,folded_pI,unfolded_pI
0,seq_D11D77826-3826-KV,4.51,5.19
1,seq_Gocatamig2_VHH1,8.68,8.46
2,seq_Envafolimab_VHH1,9.68,9.46
3,seq_Podentamig1_VHH1,9.04,9.11
4,seq_Lofacimig_VHH1,6.65,7.19


In [30]:
df.to_csv('/data/localhost/gordon/TNP_Project/RESULTS/greiff_results/folded_unfolded_pi/vhh_twist_fold_unfold_pi.csv', index=False)

-------------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------------

In [31]:
from pathlib import Path

Second part

Free energy of folding, folded charge at ph7, unfolded charge at ph7

In [49]:
dataset_name = 'vhh_twist'

In [50]:
# propka files
#propka_path = '/vols/opig/users/gordon/tnp_project/greiff_results/'+dataset_name+'_propka/'
propka_path = '/data/localhost/gordon/TNP_Project/RESULTS/greiff_results/propka/'+dataset_name+'_propka/'
propka_files = sorted([propka_path + f for f in os.listdir(propka_path) if f.endswith('.pka')])

In [51]:
len(propka_files)

108

In [52]:
def propka_data(f):

    try:
        
        with open(f) as rf:
            all_lines = [l.strip() for l in rf.readlines()] 

        #sp = [n for n, l in enumerate(all_lines) if l.startswith('SUMMARY OF THIS PREDICTION')]
        ffe = [n for n, l in enumerate(all_lines) if l.startswith('Free energy of')]
        pc = [n for n, l in enumerate(all_lines) if l.startswith('Protein charge of folded')]

        folding_free_energy = [i for i in all_lines[ffe[0]+1:pc[0]] if i and i[0].isnumeric()] # read all the lines between this section and where protein charge section starts
        ffe_ph7 = [f for f in folding_free_energy if f.startswith('7.00')][0].split()[1] # extract just value for ph7

        protein_charge = [i for i in all_lines[pc[0]+2:-1]]
        ph, pc_ph7_unfolded, pc_ph7_folded  = [p for p in protein_charge if p.startswith('7.00')][0].split()

    except Exception:
        print('PROPKA:', f.split('/')[-1][:-4]) 
        
    return {'SeqID': f.split('/')[-1][:-4], 
            'Free energy of folding':ffe_ph7,
            'Unfolded charge pH7':pc_ph7_unfolded,
            'Folded charge pH7': pc_ph7_folded}

In [53]:
all_propka_results = []
for p in propka_files:
    results = propka_data(p)
    all_propka_results.append(results)

In [54]:
len(all_propka_results)

108

In [55]:
df = pd.DataFrame(all_propka_results)#.T.reset_index()
#df.columns = ['SeqID','folded_pI', 'unfolded_pI']

In [56]:
df.to_csv('/data/localhost/gordon/TNP_Project/RESULTS/greiff_results/folded_unfolded_charge/vhh_twist_fold_unfold_charge.csv',index=False)

-------------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------------

SASA, SAP (structural aggregation propensity), DI (developability index)

In [81]:
dataset_name = 'vhh_twist'

In [82]:
# structure files (pdb) hydrogenated !!!
#structure_path = '/data/localhost/gordon/TNP_Project/results/debugged_nbb2_results/new_'+dataset_name+'_output/Raw_Model_Outputs/'
#structures = [str(s).split('/')[-1] for s in list(Path(structure_path).glob('**/*.pdb'))]

structure_path = '/data/localhost/gordon/TNP_Project/RESULTS/greiff_results/reduced_strucs/reduced_'+dataset_name+'/'
structures = os.listdir(structure_path)
structure_files = sorted([structure_path + structure for structure in structures])

In [83]:
len(structure_files)

108

In [84]:
def exposed_sasas():
    
    backbone = ['C', 'N', 'CA', 'O']
    
    exposed_sasas = defaultdict(dict)
    
    tripeptides = ['A_A_A.pdb', 'A_C_A.pdb', 'A_D_A.pdb', 'A_E_A.pdb', 
                'A_F_A.pdb', 'A_G_A.pdb', 'A_H_A.pdb', 'A_I_A.pdb',
                'A_K_A.pdb', 'A_L_A.pdb', 'A_M_A.pdb', 'A_N_A.pdb',
                'A_P_A.pdb', 'A_Q_A.pdb', 'A_R_A.pdb', 'A_S_A.pdb',
                'A_T_A.pdb', 'A_V_A.pdb', 'A_W_A.pdb', 'A_Y_A.pdb']
    
    for peptide in tripeptides:
        structure = freesasa.Structure(peptide)
        sasa = freesasa.calc(structure)
        
        for atom in range(structure.nAtoms()):
            if structure.residueNumber(atom).strip() == '3':
                data = structure.residueName(atom).strip(), structure.atomName(atom).strip(), sasa.atomArea(atom)
                exposed_sasas[data[0]].update({data[1]: data[2]})
                
    exposed_side_chain_sasas = {k: 0 for k in exposed_sasas.keys()}
    
    for aa in exposed_sasas:
        for atom in exposed_sasas[aa].keys():
            if not atom in backbone:
                exposed_side_chain_sasas[aa] += exposed_sasas[aa][atom]

    return exposed_side_chain_sasas


In [85]:
exposed_side_chain_sasas = exposed_sasas()

In [86]:
# Black, S. D., & Mould, D. R. (1991). Development of hydrophobicity parameters to analyze proteins which bear post- or cotranslational modifications. Analytical Biochemistry, 193(1), 72–82. doi:10.1016/0003-2697(91)90045-u 
hydrophobicity = {'ALA': 0.616,
                'CYS': 0.680,
                'ASP': 0.028,
                'GLU': 0.043,
                'PHE': 1.000,
                'GLY': 0.501,
                'HIS': 0.165,
                'ILE': 0.943,
                'LYS': 0.283,
                'LEU': 0.943,
                'MET': 0.738,
                'ASN': 0.236,
                'PRO': 0.711,
                'GLN': 0.251,
                'ARG': 0.000,
                'SER': 0.359,
                'THR': 0.450,
                'VAL': 0.825,
                'TRP': 0.878,
                'TYR': 0.880}

In [87]:
def sap(filename, exposed_side_chain_sasas = exposed_side_chain_sasas, hydrophobicity = hydrophobicity):
    
    def sasa(filename):

        backbone = ['C', 'N', 'CA', 'O']

        sasas = defaultdict(dict)
        sasa_int = 0

        structure = freesasa.Structure(filename)
        sasa = freesasa.calc(structure)

        for atom in range(structure.nAtoms()):
            data = structure.residueName(atom).strip(), structure.residueNumber(atom).strip(), structure.chainLabel(atom).strip(), structure.atomName(atom).strip(), sasa.atomArea(atom)
            idx = '_'.join(data[:3])
            sasas[idx].update({data[3]: data[4]})
            sasa_int += data[4]

        return sasas, sasa_int
    
    def side_chain_sasa(sasas):

        backbone = ['C', 'N', 'CA', 'O']

        side_chain_sasas = {k: 0 for k in sasas.keys()}
        side_chain_sasa_int = 0

        for res in sasas:
            for atom in sasas[res].keys():
                if not atom in backbone:
                    side_chain_sasas[res] += sasas[res][atom]
                    side_chain_sasa_int += sasas[res][atom]

        return side_chain_sasas, side_chain_sasa_int

    try:
    
        sasas = sasa(filename)
        side_chain_sasas_ = side_chain_sasa(sasas[0])
        side_chain_sasas = side_chain_sasas_[0]

        sap_values = {}

        for i in side_chain_sasas:
            for j in exposed_side_chain_sasas:
                if i.startswith(j):
                    try:
                        #print(i, j, side_chain_sasas[i] / exposed_side_chain_sasas[j])
                        sasa = side_chain_sasas[i] / exposed_side_chain_sasas[j]
                    except ZeroDivisionError:
                        if i.startswith('GLY'):
                            #print(i, j, 0)
                            sasa = 0 
                    sap = sasa * hydrophobicity[j]
                    sap_values[i] = sap

        sap_score = sum(sap_values.values())
        
        return {'SeqID':filename.split('/')[-1][:-4], 
                'SASA': sasas[1], 
                'Side chain SASA':side_chain_sasas_[1],  
                'SAP score':sap_score} # NOTE GG removed sap_values in output 
        
    except Exception:
        print(filename.split('/')[-1][:-4])
        # print('SASA:', filename.split('/')[-1][:-4])
        # sap_values = {'SER_2_L': np.inf, 'VAL_3_L': np.inf, 'LEU_4_L': np.inf}
        
        #return filename.split('/')[-1][:-4], np.inf, np.inf, sap_values, np.inf
        return filename.split('/')[-1][:-4], np.inf, np.inf, np.inf     


In [88]:
all_sap_results = []
for s in structure_files:
    #structure = s.split('/')[-1].split('.')[0]
    results = sap(s)
    all_sap_results.append(results)

In [89]:
len(all_sap_results)

108

In [90]:
df = pd.DataFrame(all_sap_results)

In [91]:
df.head()

,SeqID,SASA,Side chain SASA,SAP score
0,seq_Brivekimig1_VHH1,6126.007954,4574.552853,14.901449
1,seq_Brivekimig1_VHH2,6298.899166,4787.796926,15.188063
2,seq_Brivekimig2_VHH1,5996.372113,4322.464660,15.668135
3,seq_Caplacizumab_VHH1,6099.079770,4592.406014,14.344442
4,seq_D11D77826-1763-I,5798.427788,4300.676229,15.280289


In [92]:
df.to_csv('/data/localhost/gordon/TNP_Project/RESULTS/greiff_results/sasa_sap/vhh_twist_sasa_sap.csv',index=False)

-----------------------------------------------------------------------------------------------------------------

DI (developability index)

https://www.sciencedirect.com/science/article/pii/S0022354915317780

Using 'net charge' taken to mean folded charge of the structure at ph7

Equation for DI = SAP score - B x (net charge)^2

In [111]:
dataset_name = 'vhh_twist'

In [112]:
def get_di(charge_data, sap_data):
    
    # load in charge data
    charges = pd.read_csv(charge_data)
    # load in sap data
    saps = pd.read_csv(sap_data)
    # merge data on SeqID so using correct corresponding charge and SAP values
    df = charges.merge(saps, on='SeqID')

    beta = 0.0815 # according to Greiff code base

    di_values = []
    for charge,sap in zip(list(df['SAP score']),list(df['Folded charge pH7'])):
        di = sap - (beta * (charge)**2)
        di_values.append(di)
    
    df['DI score'] = di_values

    return df

In [113]:
charge_data = '/data/localhost/gordon/TNP_Project/RESULTS/greiff_results/folded_unfolded_charge/'+dataset_name+'_fold_unfold_charge.csv'
sap_data = '/data/localhost/gordon/TNP_Project/RESULTS/greiff_results/sasa_sap/'+dataset_name+'_sasa_sap.csv'

In [114]:
df = get_di(charge_data, sap_data)

In [115]:
len(df)

108

In [116]:
df.head()

,SeqID,Free energy of folding,Unfolded charge pH7,Folded charge pH7,SASA,Side chain SASA,SAP score,DI score
0,seq_Brivekimig1_VHH1,-12.46,2.92,2.40,6126.007954,4574.552853,14.901449,-15.697335
1,seq_Brivekimig1_VHH2,-11.33,2.92,2.49,6298.899166,4787.796926,15.188063,-16.310197
2,seq_Brivekimig2_VHH1,-13.83,1.90,1.82,5996.372113,4322.464660,15.668135,-18.187471
3,seq_Caplacizumab_VHH1,-21.73,2.93,2.84,6099.079770,4592.406014,14.344442,-13.929687
4,seq_D11D77826-1763-I,-14.53,1.92,1.78,5798.427788,4300.676229,15.280289,-17.249209


In [117]:
df.to_csv('/data/localhost/gordon/TNP_Project/RESULTS/greiff_results/dev_index/vhh_twist_di.csv',index=False)

In [118]:
# #names_1 = propka_dicts[..., 0]
# names_2 = saps[..., 0]
# #names = np.intersect1d(names_1, names_2)

# #print(len(names_1), len(names_2))

# saps_ = np.array(saps[..., -1])
# propka = np.array([propka[7.0] for propka in propka_dicts[..., 1]])

# def DI(charges, sap_score, beta = 0.0815):

#     di = {}
#     for k, v in charges.items():
#         di[k] = sap_score - beta * v[1]

#     return di

# beta = 0.0815

# dis = saps_ - beta * propka[..., 2]

# print(len(propka_dicts))

In [119]:
# df = pd.DataFrame(propka)
# df.columns = ['folding_free_energy', 'unfolded_charge', 'folded_charge']
# df[['sasas', 'side_chain_sasas']] = saps[..., 1:3]
# df['SAP'] = saps_
# df['DI'] = dis
# df.insert(0, 'ID', [s.split('/')[-1][:-4] for s in files_1])
# df.replace([np.inf, -np.inf], 0, inplace=True)
# df.to_csv('./ABB_paired_SASA_SAP_ID_2.csv', index=None)